# Proof of Concept
Ariel Silbert

Run with data collected through April 23

In [1]:
import pandas as pd
import json
import os
import numpy as np
from sklearn.decomposition import PCA, IncrementalPCA
import matplotlib.pyplot as plt
from scipy import sparse
from sklearn.feature_selection import f_regression
from sklearn.preprocessing import StandardScaler

## Load Data
- import json files
- turn into pandas dataframes
- split data into user and video information

In [2]:
#pd.read_json('TikTokImpact-main\TikTok_scrape\output\07032022_fresh\_.maur._\_.maur._.json')
#/Users/arielsilbert/Desktop/MS Data Analytics/Capstone/TikTokImpact-main/TikTok_scrape/output/07032022_fresh/_.maur._
#pd.read_json('_.maur._.json')
data = json.load(open('TikTokImpact-main/TikTok_scrape/output/07032022_fresh/_.maur._/_.maur._.json'))
df = pd.json_normalize(data)
df, data

(  itemList              user.id user.shortId user.uniqueId user.nickname  \
 0       []  6628114544067756038                   _.maur._          Maur   
 
                                    user.avatarLarger  \
 0  https://p16-sign-va.tiktokcdn.com/tos-maliva-a...   
 
                                    user.avatarMedium  \
 0  https://p16-sign-va.tiktokcdn.com/tos-maliva-a...   
 
                                     user.avatarThumb user.signature  \
 0  https://p16-sign-va.tiktokcdn.com/tos-maliva-a...           🌍🤝🇺🇦   
 
    user.createTime  ...  user.privateAccount user.secret  user.isADVirtual  \
 0       1548330054  ...                False       False             False   
 
    user.roomId  stats.followerCount  stats.followingCount  stats.heart  \
 0                              5336                     5      1400000   
 
    stats.heartCount  stats.videoCount  stats.diggCount  
 0           1400000                 3                0  
 
 [1 rows x 28 columns],
 {'user': {'

In [5]:
user_stats = pd.DataFrame()
video_stats = pd.DataFrame()

directory = 'TikTokImpact-main/TikTok_scrape/output'
for datefolder in os.listdir(directory): #go into folders by date
    f = os.path.join(directory, datefolder)
    if os.path.isdir(f): #checking if it is a directory
        for subfolder in os.listdir(f): #go into subfolders
            g = os.path.join(f, subfolder)
            if os.path.isdir(g):
                for json_file in os.listdir(g): #go into video and user stats folders
                    h = os.path.join(g, json_file)
                    if 'OrigSearch' in h:
                        data = json.load(open(h))
                        data['date'] = int(h.split('/')[3].split('_')[0]) #select date from folder name
                        df = pd.json_normalize(data)
                        video_stats = video_stats.append(df) #not an efficient way of loading the data to the df
                    else: 
                        data = json.load(open(h))
                        data['date'] = int(h.split('/')[3].split('_')[0])
                        df = pd.json_normalize(data)
                        user_stats = user_stats.append(df)
    video_stats.to_csv('video_stats.csv', index = False)
    user_stats.to_csv('user_stats.csv', index = False)

In [34]:
#apparently the video_stats file is too big to upload to github in one go
for i in range(6):
    video_stats[i*3900:(i+1)*3900].to_csv(f'video_stats_pt{i}.csv', index = False)

In [6]:
video_stats.iloc[:,:]
#numerical: createTime, itemCommentStatus, video.duration, video.bitrate, 
#music.duration, stats.diggCount, stats.shareCount, stats.commentCount, 
#stats.playCount, authorStats.followingCount, authorStats.followerCount, 
#authorStats.heartCount, authorStats.videoCount, authorStats.diggCount, authorStats.heart

,id,desc,createTime,challenges,originalItem,officalItem,textExtra,secret,forFriend,digged,...,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount,authorStats.heart,maskType,effectStickers,author.roomId,warnInfo
0,7069787462242962694,#ukraine #russia,1646063166,"[{'id': '451042', 'title': 'ukraine', 'desc': ...",False,False,"[{'awemeId': '', 'start': 0, 'end': 8, 'hashta...",False,False,False,...,0,165100,4700000,910,548,4700000,NaN,NaN,NaN,NaN
0,7082556143888878890,#ukraine #ukrainevsrussia #ukrainewar #bucha,1649036106,"[{'id': '451042', 'title': 'ukraine', 'desc': ...",False,False,"[{'awemeId': '', 'start': 0, 'end': 8, 'hashta...",False,False,False,...,4,204,1121,145,135,1121,3.0,NaN,NaN,NaN
0,7068388771854634245,I wish this is a bad dream and we will wake up...,1645737509,"[{'id': '861413', 'title': 'stopwar', 'desc': ...",False,False,"[{'awemeId': '', 'start': 56, 'end': 64, 'hash...",False,False,False,...,63,124400,3100000,69,61,3100000,NaN,NaN,NaN,NaN
0,7082576425911323946,its exciting every timeeee,1649040828,NaN,False,False,NaN,False,False,False,...,6098,1652,18800,1662,84700,18800,NaN,NaN,NaN,NaN
0,7071700963953741062,🇺🇦🤝 🇷🇺 #ukraine #supportukraine #fyp #nowar,1646508688,"[{'id': '451042', 'title': 'ukraine', 'desc': ...",False,False,"[{'awemeId': '', 'start': 12, 'end': 20, 'hash...",False,False,False,...,32,10800000,226600000,701,3361,226600000,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,7070296231565692161,#foryourpage #vladimirputin #shocking #murtyme...,1646181627,"[{'id': '1598498371111942', 'title': 'foryourp...",False,False,"[{'awemeId': '', 'start': 0, 'end': 12, 'hasht...",False,False,False,...,0,187600,2400000,250,335,2400000,NaN,NaN,NaN,NaN
0,7072706328795368709,Le présentent ukrainien dénonce l'attaque cont...,1646742768,"[{'id': '1494657', 'title': 'wwiii', 'desc': '...",False,False,"[{'awemeId': '', 'start': 56, 'end': 62, 'hash...",False,False,False,...,10,71,1028,7,66,1028,NaN,NaN,NaN,NaN
0,7068763810110164270,Idk how else to say it #ukraine #russia #ww3,1645824828,"[{'id': '451042', 'title': 'ukraine', 'desc': ...",False,False,"[{'awemeId': '', 'start': 23, 'end': 31, 'hash...",False,False,False,...,895,1301,242600,300,7889,242600,NaN,NaN,NaN,NaN
0,7068420650159017262,Dayum #fyp #xyzbca #ukraine #russia,1645744930,"[{'id': '229207', 'title': 'fyp', 'desc': '', ...",False,False,"[{'awemeId': '', 'start': 6, 'end': 10, 'hasht...",False,False,False,...,329,42300,3300000,116,644,3300000,NaN,NaN,NaN,NaN


In [7]:
user_stats.fillna(0)
user_stats.iloc[:,20:]
#numerical columns: 'stats.followerCount', 'stats.followingCount', 'stats.heart', 'stats.heartCount', 'stats.videoCount', 'stats.diggCount'
#Emily to look into what createTime means IRL
#user.relation appears to be all 0s -- unclear what it is
#user.bioLink.risk probably a cybersecurity thing
#user...Setting seem to be T/F values but as numbers?
#diggs used to be likes -- field may not be used anymore, but still exists

,user.stitchSetting,user.privateAccount,user.secret,user.isADVirtual,user.roomId,stats.followerCount,stats.followingCount,stats.heart,stats.heartCount,stats.videoCount,stats.diggCount,user.commerceUserInfo.commerceUser,user.commerceUserInfo.category,user.commerceUserInfo.categoryButton,user.commerceUserInfo.downLoadLink.android,user.commerceUserInfo.downLoadLink.ios,user.showPrivateBanner
0,0.0,False,False,False,,165100.0,0.0,4700000.0,4700000.0,910.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,0.0,False,False,False,,204.0,4.0,1121.0,1121.0,145.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,0.0,False,False,False,,124400.0,63.0,3100000.0,3100000.0,69.0,0.0,True,Personal Blog,False,NaN,NaN,NaN
0,0.0,False,False,False,,1652.0,6098.0,18800.0,18800.0,1662.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,0.0,False,False,False,,10800000.0,32.0,226600000.0,226600000.0,701.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,3.0,False,False,False,,187600.0,0.0,2400000.0,2400000.0,250.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,0.0,False,False,False,,71.0,10.0,1028.0,1028.0,7.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,0.0,False,False,False,,1301.0,895.0,242600.0,242600.0,300.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
0,0.0,False,False,False,,42300.0,329.0,3300000.0,3300000.0,116.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN


## Preliminary Statistics
- correlations
- description (mean, std, count, min, max, 25th and 75th %iles)

In [8]:
video_stats.corr().dropna(how='all').dropna(axis=1).iloc[:,:13]

,createTime,duetEnabled,stitchEnabled,date,video.duration,video.bitrate,author.stitchSetting,music.original,music.duration,stats.diggCount,stats.shareCount,stats.commentCount,stats.playCount
createTime,1.000000,-0.042621,-0.082326,0.027976,-0.016843,-0.021221,0.099411,-0.005863,-0.001959,-0.208728,-0.150938,-0.169744,-0.226708
itemCommentStatus,0.029116,-0.085241,-0.082957,-0.010298,-0.023353,-0.040401,0.088502,-0.055152,0.008814,-0.041327,-0.031758,-0.064274,0.010850
duetEnabled,-0.042621,1.000000,0.825618,0.017660,-0.298876,0.040001,-0.295167,0.002775,-0.218807,-0.020370,-0.062157,-0.064404,-0.048756
stitchEnabled,-0.082326,0.825618,1.000000,0.017963,-0.246029,0.046296,-0.263864,0.002533,-0.181193,-0.002354,-0.038635,-0.021150,-0.036343
date,0.027976,0.017660,0.017963,1.000000,-0.018751,0.005747,0.009493,-0.001563,-0.013604,-0.034196,-0.026916,-0.032281,-0.038464
video.height,-0.043187,0.044653,0.047052,-0.023532,0.068440,0.133934,-0.027485,-0.056923,0.057091,0.037691,0.032204,0.013642,0.029110
video.width,0.033337,-0.043006,-0.031335,-0.018245,0.070575,0.072087,0.023129,0.006676,0.023747,-0.048142,-0.049789,-0.041900,-0.066341
video.duration,-0.016843,-0.298876,-0.246029,-0.018751,1.000000,-0.105105,-0.036556,-0.039016,0.686298,0.039780,0.174949,0.043314,0.058061
video.bitrate,-0.021221,0.040001,0.046296,0.005747,-0.105105,1.000000,-0.014631,-0.055697,-0.081560,0.093263,0.020164,0.058157,0.078349
author.verified,-0.102834,-0.017283,0.041561,0.042344,0.166687,-0.074742,-0.129818,0.108843,0.076643,0.028609,0.096632,0.091445,0.096708


In [9]:
video_stats.corr().dropna(how='all').dropna(axis=1).iloc[:,13:]

,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount,authorStats.heart
createTime,0.030845,-0.126420,-0.098546,-0.049166,-0.038340,-0.098546
itemCommentStatus,0.015420,-0.046224,-0.035905,-0.024001,-0.021127,-0.035905
duetEnabled,-0.040957,0.027663,-0.002906,-0.036808,0.076063,-0.002906
stitchEnabled,-0.055364,0.067409,0.031208,-0.001696,0.099710,0.031208
date,0.010535,0.039648,0.038690,0.032673,0.008974,0.038690
video.height,-0.023603,0.051482,0.030064,0.055559,0.024534,0.030064
video.width,0.024792,-0.001361,0.002560,-0.010827,-0.009951,0.002560
video.duration,-0.041741,0.123579,0.053854,0.055312,-0.090197,0.053854
video.bitrate,0.000553,-0.017561,-0.064075,-0.077927,0.067344,-0.064075
author.verified,-0.049277,0.533290,0.431921,0.457954,-0.107766,0.431921


In [10]:
video_stats.describe().iloc[:,:13]

,createTime,itemCommentStatus,date,video.height,video.width,video.duration,video.bitrate,author.relation,author.commentSetting,author.duetSetting,author.stitchSetting,music.duration,stats.diggCount
count,1.987000e+04,19870.000000,1.987000e+04,19870.000000,19870.000000,19870.000000,1.979700e+04,19870.0,19870.000000,19870.000000,19870.000000,19868.000000,1.987000e+04
mean,1.646293e+09,0.039104,1.611756e+07,986.748968,574.816608,27.443533,1.032938e+06,0.0,0.011374,0.198842,0.237494,38.072428,7.358321e+05
std,2.663534e+06,0.340278,7.836725e+06,142.717635,72.411486,30.759785,6.881344e+05,0.0,0.152723,0.707920,0.774560,40.286891,1.024170e+06
min,1.578792e+09,0.000000,1.042022e+06,0.000000,0.000000,0.000000,5.501600e+04,0.0,0.000000,0.000000,0.000000,2.000000,0.000000e+00
25%,1.645737e+09,0.000000,1.004202e+07,1024.000000,576.000000,10.000000,5.749310e+05,0.0,0.000000,0.000000,0.000000,14.000000,1.823500e+05
50%,1.645953e+09,0.000000,1.703202e+07,1024.000000,576.000000,17.000000,8.615830e+05,0.0,0.000000,0.000000,0.000000,27.000000,3.816000e+05
75%,1.646958e+09,0.000000,2.104202e+07,1024.000000,576.000000,31.000000,1.299922e+06,0.0,0.000000,0.000000,0.000000,58.000000,9.031000e+05
max,1.650577e+09,3.000000,3.103202e+07,1344.000000,1332.000000,600.000000,8.313859e+06,0.0,3.000000,3.000000,3.000000,981.000000,1.070000e+07


In [11]:
video_stats.describe().iloc[:,13:]

,stats.shareCount,stats.commentCount,stats.playCount,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount,authorStats.heart,maskType
count,19870.000000,19870.000000,1.987000e+04,19870.000000,1.987000e+04,1.987000e+04,19870.000000,19870.000000,1.987000e+04,8.00000
mean,19185.835682,12236.641671,6.423756e+06,563.093306,5.560369e+05,1.888577e+07,425.050629,14747.706140,1.888577e+07,3.75000
std,36817.407433,21879.462847,8.034207e+06,1382.071238,1.167972e+06,5.603230e+07,813.337660,35223.415263,5.603230e+07,0.46291
min,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,1.000000,0.000000,0.000000e+00,3.00000
25%,1143.250000,1181.250000,1.200000e+06,21.000000,6.253250e+03,3.235250e+05,31.000000,330.000000,3.235250e+05,3.75000
50%,6272.000000,5814.500000,4.000000e+06,112.000000,6.010000e+04,2.000000e+06,124.000000,2554.500000,2.000000e+06,4.00000
75%,19900.000000,15175.000000,8.800000e+06,542.000000,5.726750e+05,1.220000e+07,426.000000,12375.000000,1.220000e+07,4.00000
max,533400.000000,362500.000000,7.960000e+07,10000.000000,2.190000e+07,2.000000e+09,11100.000000,358300.000000,2.000000e+09,4.00000


In [12]:
user_stats.describe()

,date,user.createTime,user.relation,user.bioLink.risk,user.commentSetting,user.duetSetting,user.stitchSetting,stats.followerCount,stats.followingCount,stats.heart,stats.heartCount,stats.videoCount,stats.diggCount
count,1.639900e+04,1.362900e+04,16326.0,4968.000000,16326.000000,16326.000000,16326.000000,1.632600e+04,16326.000000,1.632600e+04,1.632600e+04,16326.000000,16326.000000
mean,1.589983e+07,1.596278e+09,0.0,1.935990,0.012679,0.234044,0.275695,3.517773e+05,608.430234,1.130337e+07,1.130337e+07,314.641308,508.101127
std,7.768775e+06,4.342350e+07,0.0,1.505218,0.159320,0.761988,0.827109,1.074732e+06,1470.525694,5.195411e+07,5.195411e+07,638.104142,5712.605420
min,1.042022e+06,1.436840e+09,0.0,0.000000,0.000000,0.000000,0.000000,0.000000e+00,0.000000,0.000000e+00,0.000000e+00,0.000000,0.000000
25%,1.004202e+07,1.574040e+09,0.0,0.000000,0.000000,0.000000,0.000000,3.411250e+03,25.000000,2.149500e+05,2.149500e+05,26.000000,0.000000
50%,1.703202e+07,1.599663e+09,0.0,3.000000,0.000000,0.000000,0.000000,2.965000e+04,127.000000,1.200000e+06,1.200000e+06,84.000000,0.000000
75%,2.104202e+07,1.631311e+09,0.0,3.000000,0.000000,0.000000,0.000000,2.027500e+05,505.000000,4.900000e+06,4.900000e+06,302.000000,0.000000
max,3.103202e+07,1.649861e+09,0.0,5.000000,3.000000,3.000000,3.000000,2.190000e+07,10000.000000,2.000000e+09,2.000000e+09,11100.000000,258800.000000


## PCA
- used imcremental PCA because of the size of the dataset

In [28]:
video_stats[v_cols].dropna()

,video.duration,music.duration,stats.diggCount,stats.shareCount,stats.commentCount,stats.playCount,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount
0,20,60.0,863800,78200,11400,6700000,0,165100,4700000,910,548
0,104,104.0,152,8,15,6913,4,204,1121,145,135
0,9,11.0,2300000,18900,82000,14700000,63,124400,3100000,69,61
0,15,60.0,4,0,0,17,6098,1652,18800,1662,84700
0,42,42.0,1900000,27400,44700,10500000,32,10800000,226600000,701,3361
...,...,...,...,...,...,...,...,...,...,...,...
0,17,60.0,731500,6125,0,22500000,0,187600,2400000,250,335
0,22,22.0,554,10,60,29300,10,71,1028,7,66
0,9,9.0,232600,2236,2957,1300000,895,1301,242600,300,7889
0,12,45.0,276900,3800,4215,1000000,329,42300,3300000,116,644


In [29]:
v_cols = ['video.duration', 'music.duration', 'stats.diggCount', 'stats.shareCount', 'stats.commentCount', 
          'stats.playCount', 'authorStats.followingCount', 'authorStats.followerCount', 'authorStats.heartCount', 
          'authorStats.videoCount', 'authorStats.diggCount']
X = video_stats[v_cols].dropna()
X = StandardScaler().fit_transform(X)

pca = IncrementalPCA(n_components=6, batch_size = 100)

X_sparse = sparse.csr_matrix(X)
X_transformed = pca.fit_transform(X_sparse)
X_transformed.shape

print(pca.explained_variance_ratio_, sum(pca.explained_variance_ratio_))
print(pca.singular_values_)

[0.28633463 0.21831452 0.15393693 0.11247455 0.06406454 0.03999624] 0.8751214170652954
[250.15567512 218.43122846 183.41921711 156.7835693  118.32656531
  93.49384596]


In [30]:
PC_nums = []
for i in range(6):
    PC_nums.append(f'PC {i+1}')

pd.DataFrame(pca.components_,columns=v_cols,index = PC_nums)

,video.duration,music.duration,stats.diggCount,stats.shareCount,stats.commentCount,stats.playCount,authorStats.followingCount,authorStats.followerCount,authorStats.heartCount,authorStats.videoCount,authorStats.diggCount
PC 1,0.107312,0.060392,0.496373,0.447914,0.465130,0.496343,-0.050569,0.198575,0.147479,0.104760,-0.020191
PC 2,0.099519,0.097596,-0.136095,-0.120568,-0.131015,-0.135685,0.182654,0.532957,0.567129,0.518510,0.063586
PC 3,0.670530,0.673252,-0.087364,0.042511,-0.077286,-0.064038,-0.144679,-0.049712,-0.101110,-0.067758,-0.198467
PC 4,0.125651,0.185814,0.045955,0.099437,0.039141,-0.023678,0.648440,-0.142784,-0.117370,-0.058246,0.691060
PC 5,-0.023666,-0.050077,-0.064191,0.130689,0.059389,-0.001729,0.689236,-0.126171,-0.089855,0.110494,-0.678936
PC 6,-0.245672,0.232621,-0.118957,0.107898,0.231435,-0.114230,-0.188764,-0.418402,-0.200556,0.732437,0.075829


In [16]:
u_cols = ['stats.followerCount', 'stats.followingCount', 'stats.heart', 'stats.videoCount', 'stats.diggCount']
X2 = user_stats[u_cols]
X2 = X2.dropna()

X2 = StandardScaler().fit_transform(X2)

pca2 = IncrementalPCA(n_components=3, batch_size = 200)

X2_sparse = sparse.csr_matrix(X2)
X2_transformed = pca2.fit_transform(X2_sparse)
X2_transformed.shape

print(pca2.explained_variance_ratio_, sum(pca2.explained_variance_ratio_))
print(pca2.singular_values_)

[0.46441916 0.22351996 0.17198   ] 0.859919121384738
[194.70628173 135.07751204 118.48513609]


In [17]:
PC_nums = []
for i in range(3):
    PC_nums.append(f'PC {i+1}')

pd.DataFrame(pca2.components_,columns=u_cols,index = PC_nums)

,stats.followerCount,stats.followingCount,stats.heart,stats.videoCount,stats.diggCount
PC 1,0.588486,0.163696,0.591602,0.526193,0.004139
PC 2,-0.159176,0.654065,-0.111438,0.094133,0.724971
PC 3,-0.120329,0.713331,-0.085484,0.014158,-0.684961


## F Regression

In [31]:
for i in range(len(v_cols)):
    print(f'y = {v_cols[i]} \nf statistic: {f_regression(X, X[:,i])[0]} \np values: {f_regression(X, X[:,i])[1]}')

y = video.duration 
f statistic: [-3.82344061e+17  1.76881997e+04  3.14867344e+01  6.27255820e+02
  3.73410978e+01  6.71974944e+01  3.46793530e+01  3.08097247e+02
  5.77837305e+01  6.09654307e+01  1.62945186e+02] 
p values: [1.00000000e+000 0.00000000e+000 2.03484827e-008 2.56156497e-136
 1.01019623e-009 2.60390993e-016 3.94994110e-009 1.86445745e-068
 3.05550599e-014 6.09540905e-015 3.59986287e-037]
y = music.duration 
f statistic: [ 1.76881997e+04 -8.77142257e+17  1.18837830e-01  1.38903370e+02
  1.92196892e+00  1.28528646e-03  1.19070460e+00  8.01590145e+01
  1.22312410e+01  8.59359117e+01  3.69744744e+01] 
p values: [0.00000000e+00 1.00000000e+00 7.30302040e-01 5.90874146e-32
 1.65655329e-01 9.71401613e-01 2.75200710e-01 3.75222527e-19
 4.70985958e-04 2.04347590e-20 1.21872194e-09]
y = stats.diggCount 
f statistic: [ 3.14867344e+01  1.18837830e-01 -2.23671275e+19  1.26345157e+04
  1.90461586e+04  4.91734844e+04  1.51124781e+02  3.83363962e+02
  6.41727211e+01  2.33970297e+00  1.455

In [19]:
for i in range(len(u_cols)):
    print(f'y = {u_cols[i]} \nf statistics: {f_regression(X2, X2[:,i])[0]}, \np values: {f_regression(X2, X2[:,i])[1]}')

y = stats.followerCount 
f statistics: [-3.06319835e+18  1.00417995e+02  3.22821256e+04  6.92508847e+03
  1.15246268e+01], 
p values: [1.00000000e+00 1.44358464e-23 0.00000000e+00 0.00000000e+00
 6.88436973e-04]
y = stats.followingCount 
f statistics: [1.00417995e+02 2.10047887e+18 2.51456362e+02 6.19277085e+02
 2.54867524e+02], 
p values: [1.44358464e-023 0.00000000e+000 3.28504915e-056 3.36579010e-134
 6.08514752e-057]
y = stats.heart 
f statistics: [ 3.22821256e+04  2.51456362e+02 -9.18959504e+18  7.33478804e+03
  4.77299056e+00], 
p values: [0.00000000e+00 3.28504915e-56 1.00000000e+00 0.00000000e+00
 2.89237382e-02]
y = stats.videoCount 
f statistics: [6.92508847e+03 6.19277085e+02 7.33478804e+03 1.13102708e+18
 1.05453032e+01], 
p values: [0.0000000e+000 3.3657901e-134 0.0000000e+000 0.0000000e+000
 1.1671887e-003]
y = stats.diggCount 
f statistics: [ 1.15246268e+01  2.54867524e+02  4.77299056e+00  1.05453032e+01
 -8.35417731e+17], 
p values: [6.88436973e-04 6.08514752e-57 2.8923